<a href="https://colab.research.google.com/github/ALMerrill/cs474_labs_f2019/blob/master/DL_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/ALMerrill/cs474_labs_f2019/blob/master/DL_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4: Cancer Detection

## Objective
* To build a dense prediction model
* To begin reading current papers in DNN research

## Deliverable
For this lab, you will turn in a notebook that describes your efforts at creating
a pytorch radiologist. Your final deliverable is a notebook that has (1) deep network,
(2) cost function, (3) method of calculating accuracy,
(4) an image that shows the dense prediction produced by your network on the pos_test_000072.png image.
This is an image in the test set that your network will not have seen before.
This image, and the ground truth labeling, is shown below.
(And is contained in the downloadable dataset below).

![](http://liftothers.org/dokuwiki/lib/exe/fetch.php?w=200&tok=a8ac31&media=cs501r_f2016:pos_test_000072_output.png)
<img src="http://liftothers.org/dokuwiki/lib/exe/fetch.php?media=cs501r_f2016:pos_test_000072.png" width="200">


## Grading standards
Your notebook will be graded on the following:
* 40% Proper design, creation and debugging of a dense prediction network
* 40% Proper implementation of a loss function and train/test set accuracy measure
* 10% Tidy visualizations of loss of your dense predictor during training
* 10% Test image output


## Data set
The data is given as a set of 1024×1024 PNG images. Each input image (in 
the ```inputs``` directory) is an RGB image of a section of tissue,
and there a file with the same name (in the ```outputs``` directory) 
that has a dense labeling of whether or not a section of tissue is cancerous
(white pixels mean “cancerous”, while black pixels mean “not cancerous”).

The data has been pre-split for you into test and training splits.
Filenames also reflect whether or not the image has any cancer at all 
(files starting with ```pos_``` have some cancerous pixels, while files 
starting with ```neg_``` have no cancer anywhere).
All of the data is hand-labeled, so the dataset is not very large.
That means that overfitting is a real possibility.

## Description
For a video including some tips and tricks that can help with this lab: [https://youtu.be/Ms19kgK_D8w](https://youtu.be/Ms19kgK_D8w)
For this lab, you will implement a virtual radiologist.
You are given images of possibly cancerous tissue samples, 
and you must build a detector that identifies where in the tissue cancer may reside.

---

### Part 0
Watch and follow video tutorial:

https://youtu.be/Ms19kgK_D8w

**TODO:**

* Watch tutorial

**DONE:**

### Part 1
Implement a dense predictor

In previous labs and lectures, we have talked about DNNs that classify an 
entire image as a single class. Here, however, we are interested in a more nuanced classification: 
given an input image, we would like to identify each pixel that is possibly cancerous. 
That means that instead of a single output, your network should output an “image”, 
where each output pixel of your network represents the probability that a pixel is cancerous.

**TODO:**

* Create a Network that classifies each pixel as a 1 or 0 for cancerous / not cancerous

**DONE:**

___

### Part 1a
Implement your network topology


Use the “Deep Convolution U-Net” from this paper: [(U-Net: Convolutional Networks for Biomedical Image Segmentation)](https://arxiv.org/pdf/1505.04597.pdf) 

![(Figure 1)](https://lh3.googleusercontent.com/qnHiB3B2KRxC3NjiSDtY08_DgDGTDsHcO6PP53oNRuct-p2QXCR-gyLkDveO850F2tTAhIOPC5Ha06NP9xq1JPsVAHlQ5UXA5V-9zkUrJHGhP_MNHFoRGnjBz1vn1p8P2rMWhlAb6HQ=w2400)

You should use existing pytorch functions (not your own Conv2D module), such as ```nn.Conv2d```;
you will also need the pytorch function ```torch.cat``` and ```nn.ConvTranspose2d```

```torch.cat``` allows you to concatenate tensors.
```nn.ConvTranspose2d``` is the opposite of ```nn.Conv2d```.
It is used to bring an image from low res to higher res.
[This blog](https://towardsdatascience.com/up-sampling-with-transposed-convolution-9ae4f2df52d0) should help you understand this function in detail.

Note that the simplest network you could implement (with all the desired properties)
is just a single convolution layer with two filters and no relu! 
Why is that? (of course it wouldn't work very well!)

**TODO:**

* Understand the U-Net architecture
* Understand concatenation of inputs from multiple prior layers
* Understand ConvTranspose
* Answer Question / Reflect on simplest network with the desired properties

**DONE:**


___
The intention of this lab is to learn how to make deep neural nets and implement loss function.
Therefore we'll help you with the implementation of Dataset.
This code will download the dataset for you so that you are ready to use it and focus on network
implementation, losses and accuracies.

In [0]:
# !pip3 install torch
# !pip3 install torchvision
# !pip3 install tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
from torch.nn.parameter import Parameter
import pdb
import torchvision
import os
import gzip
import tarfile
import gc
from IPython.core.ultratb import AutoFormattedTB
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

assert torch.cuda.is_available(), "You need to request a GPU from Runtime > Change Runtime"

In [0]:
#log max trick (also picture on phone)
#     maxes, _ = torch.max(y_hat, 1, keepdim=True)
#     y_hat = y_hat - maxes
#     y_hat = torch.exp(y_hat)
#     y_hat = y_hat / torch.sum(y_hat, 1, keepdim=True)


class CancerDataset(Dataset):
  def __init__(self, root, download=True, size=512, train=True):
    if download and not os.path.exists(os.path.join(root, 'cancer_data')):
#       datasets.utils.download_url('http://liftothers.org/cancer_data.tar.gz', root, 'cancer_data.tar.gz', None)
      datasets.utils.download_url('https://nolans-cs-bucket.s3-us-west-1.amazonaws.com/cancer_data.tar.gz', root, 'cancer_data.tar.gz', None)
      self.extract_gzip(os.path.join(root, 'cancer_data.tar.gz'))
      self.extract_tar(os.path.join(root, 'cancer_data.tar'))
    
    postfix = 'train' if train else 'test'
    root = os.path.join(root, 'cancer_data', 'cancer_data')
    self.dataset_folder = torchvision.datasets.ImageFolder(os.path.join(root, 'inputs_' + postfix) ,transform = transforms.Compose([transforms.Resize(size),transforms.ToTensor()]))
    self.label_folder = torchvision.datasets.ImageFolder(os.path.join(root, 'outputs_' + postfix) ,transform = transforms.Compose([transforms.Resize(size),transforms.ToTensor()]))

  @staticmethod
  def extract_gzip(gzip_path, remove_finished=False):
    print('Extracting {}'.format(gzip_path))
    with open(gzip_path.replace('.gz', ''), 'wb') as out_f, gzip.GzipFile(gzip_path) as zip_f:
      out_f.write(zip_f.read())
    if remove_finished:
      os.unlink(gzip_path)
  
  @staticmethod
  def extract_tar(tar_path):
    print('Untarring {}'.format(tar_path))
    z = tarfile.TarFile(tar_path)
    z.extractall(tar_path.replace('.tar', ''))

  def __getitem__(self,index):
    img = self.dataset_folder[index]
    label = self.label_folder[index]
    return img[0],label[0][0]
  
  def __len__(self):
    return 100
#     return len(self.dataset_folder)

___

### Part 1b
Implement a cost function

You should still use cross-entropy as your cost function, but you may need to think hard about how exactly to set this up – your network should output cancer/not-cancer probabilities for each pixel, which can be viewed as a two-class classification problem.

**TODO:**

* Adapt CrossEntropyLoss for 2 class pixel classification

**DONE:**



In [0]:
# You'll probably want a function or something to test input / output sizes of the ConvTranspose2d layer


In [0]:
# Since you will be using the output of one network in two places(convolution and maxpooling),
# you can't use nn.Sequential.
# Instead you will write up the network like normal variable assignment as the example shown below:
# You are welcome (and encouraged) to use the built-in batch normalization and dropout layer.

# TODO: You need to change this to fit the UNet structure!!!

class ConvBlock(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(ConvBlock, self).__init__()
    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = 1, padding = 1)
    self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1)
#     self.poolConv = nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 2, padding = 1)
    
  def forward(self, x):
    out1 = F.relu(self.conv1(x))
    out2 = F.relu(self.conv2(out1))
    return out2
#     return F.relu(self.poolConv(out2))  #probably don't want relu on final layer
  
  
class ConvTransposeBlock(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(ConvTransposeBlock, self).__init__()
    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = 1, padding = 1)
    self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1)
    self.convT1 = nn.ConvTranspose2d(out_channels, out_channels // 2, kernel_size = 4, stride = 2, padding = 1) #Is it fine to just divide the channels by 2? That way it comes out to the same when the other gets concatted on
    
  def forward(self, x, skip_connection=torch.Tensor([]).cuda(), last_layer=False):
    print('ConvTransposeBlock')
#     print('skip:',skip_connection.size())
#     print('before cat:',x.size())
    x = torch.cat((skip_connection, x), 1)
#     print('after_cat:',x.size())
    out1 = F.relu(self.conv1(x))
#     print('out1:',out1.size())
    out2 = F.relu(self.conv2(out1))
    print('out2:',out2.size())
    if last_layer:
      return out2
    out3 = F.relu(self.convT1(out2))
    print('out3:',out3.size())
    return out3
      

  
class CancerDetection(nn.Module): #lab video at 39:00 for a bit of a description of the layout for these
  def __init__(self):
    super(CancerDetection, self).__init__()
    self.block1 = ConvBlock(3, 64)
    self.pool1 = nn.Conv2d(64, 64, kernel_size = 3, stride = 2, padding = 1)
    self.block2 = ConvBlock(64, 128)
    self.pool2 = nn.Conv2d(128, 128, kernel_size = 3, stride = 2, padding = 1)
    self.block3 = ConvBlock(128, 256)
    self.pool3 = nn.Conv2d(256, 256, kernel_size = 3, stride = 2, padding = 1)
    self.block4 = ConvBlock(256, 512)
    self.pool4 = nn.Conv2d(512, 512, kernel_size = 3, stride = 2, padding = 1)
    #UpConv
    self.block5 = ConvTransposeBlock(512, 1024)
    self.block6 = ConvTransposeBlock(1024, 512)
    self.block7 = ConvTransposeBlock(512, 256)
    self.block8 = ConvTransposeBlock(256, 128)
    self.block9 = ConvTransposeBlock(128, 64)
    self.conv1x1 = nn.Conv2d(64, 2, kernel_size=1, stride=1)

    
 
  def forward(self, x):
    print('\nshapes')
    print('x:',x.size())
    out1 = self.block1(x)
    pool1 = self.pool1(out1)
    print('out1:',out1.size())
    print('pool1:',pool1.size())
    out2 = self.block2(pool1)
    pool2 = self.pool2(out2)
    print('out2:',out2.size())
    out3 = self.block3(pool2)
    pool3 = self.pool3(out3)
    print('out3:',out3.size())
    out4 = self.block4(pool3)
    pool4 = self.pool4(out4)
    print('out4:',out4.size())
    #UpConv
    out5 = self.block5(pool4)
    out6 = self.block6(out5, skip_connection=out4)
    out7 = self.block7(out6, skip_connection=out3)
    out8 = self.block8(out7, skip_connection=out2)
    out9 = self.block9(out8, skip_connection=out1, last_layer=True)
    out10 = self.conv1x1(out9) #Make sure it is back to image size before this
    return out10
 


In [0]:
# Create your datasets and neural network as you have before

train_dataset = CancerDataset('root')
val_dataset = CancerDataset('root', train=False)
model = CancerDetection()
model.cuda()
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

batch_size = 4

train_loader = DataLoader(train_dataset,
                         batch_size=batch_size,
                         num_workers =4, #check iterations/sec in tqdm to see how this helps or hurts
                         pin_memory=True)
val_loader = DataLoader(val_dataset,
                       batch_size=batch_size,
                       num_workers =4,
                       pin_memory=True)

data_loaders = {'train': train_loader, 'val': val_loader}

losses = []
validations = []
train_acc = []
val_acc = []

In [100]:
# This is what was talked about in the video for memory management

def accuracy(y_hat, y_truth):
  # y_hat: batch x 2 x 512 x 512   Each channel holds the probabilities for that class
  # y_truth: batch x 1 x 512 x 512
  y_hat = y_hat[0]
  y_truth = y_truth[0]
  y_hat[0] = y_hat[0] > .5
  y_hat[1] = y_hat[1] >= .5
  intersection = torch.sum(y_hat[0] * y_truth)
  #union = torch.sum(y_hat[0] + y_truth >= 1)  #from help session?
  union = torch.sum(y_hat[0] + y_truth) - intersection
  return intersection / union
  

EPOCHS = 1

def train():
  for epoch in range(EPOCHS):
    loop = tqdm(total=len(train_loader), position=0, leave=False)
  
    for batch, (x, y_truth) in enumerate(train_loader):
      x, y_truth = x.cuda(async=True), y_truth.cuda(async=True)
      optimizer.zero_grad()
      y_hat = model(x) 
      print(y_hat.size(), y_truth.size())
      print(y_truth.type())
      loss = objective(y_hat, y_truth.long())
      loss.backward()

      losses.append(loss.item())
      acc = accuracy(y_hat, y_truth)
      train_acc.append(acc)
      mem = torch.cuda.memory_allocated(0) / 1e9

      loop.set_description('epoch: {}, loss: {:.4f}, accuracy: {:.3f}, mem: {:.2f}'
                           .format(epoch, loss, acc, mem))
      loop.update(1)
      optimizer.step()
      if batch % 20 == 0:
        y_hat_val = np.mean([accuracy(model(x.cuda()), y_truth.cuda()) for x, y_truth in val_loader])
        val_acc.append((len(losses), y_hat_val))
        val = np.mean([objective(model(x.cuda()), y.cuda()).item() 
                       for x, y in val_loader])
        validations.append((len(losses), val))
    loop.close()

def scope():
  try:
    #your code for calling dataset and dataloader
    gc.collect()
    print(torch.cuda.memory_allocated(0) / 1e9)
    
    train()
    # Call your model, figure out loss and accuracy
    
  except:
    __ITB__()
    
scope()


  0%|          | 0/25 [00:00<?, ?it/s]

0.354602496

shapes
x: torch.Size([4, 3, 512, 512])
out1: torch.Size([4, 64, 512, 512])
pool1: torch.Size([4, 64, 256, 256])
out2: torch.Size([4, 128, 256, 256])
out3: torch.Size([4, 256, 128, 128])
out4: torch.Size([4, 512, 64, 64])
ConvTransposeBlock
out2: torch.Size([4, 1024, 32, 32])
out3: torch.Size([4, 512, 64, 64])
ConvTransposeBlock
out2: torch.Size([4, 512, 64, 64])
out3: torch.Size([4, 256, 128, 128])
ConvTransposeBlock
out2: torch.Size([4, 256, 128, 128])
out3: torch.Size([4, 128, 256, 256])
ConvTransposeBlock
out2: torch.Size([4, 128, 256, 256])
out3: torch.Size([4, 64, 512, 512])
ConvTransposeBlock
out2: torch.Size([4, 64, 512, 512])
torch.Size([4, 2, 512, 512]) torch.Size([4, 512, 512])
torch.cuda.FloatTensor


epoch: 0, loss: 0.8060, accuracy: nan, mem: 0.38:   4%|▍         | 1/25 [00:04<01:48,  4.51s/it]


shapes
x: torch.Size([4, 3, 512, 512])
out1: torch.Size([4, 64, 512, 512])
pool1: torch.Size([4, 64, 256, 256])
out2: torch.Size([4, 128, 256, 256])
out3: torch.Size([4, 256, 128, 128])
out4: torch.Size([4, 512, 64, 64])
ConvTransposeBlock
out2: torch.Size([4, 1024, 32, 32])
out3: torch.Size([4, 512, 64, 64])
ConvTransposeBlock
out2: torch.Size([4, 512, 64, 64])
out3: torch.Size([4, 256, 128, 128])
ConvTransposeBlock
out2: torch.Size([4, 256, 128, 128])
out3: torch.Size([4, 128, 256, 256])
ConvTransposeBlock
out2: torch.Size([4, 128, 256, 256])
out3: torch.Size([4, 64, 512, 512])
ConvTransposeBlock
out2: torch.Size([4, 64, 512, 512])

shapes
x: torch.Size([4, 3, 512, 512])
out1: torch.Size([4, 64, 512, 512])
pool1: torch.Size([4, 64, 256, 256])
out2: torch.Size([4, 128, 256, 256])
out3: torch.Size([4, 256, 128, 128])
out4: torch.Size([4, 512, 64, 64])
ConvTransposeBlock
out2: torch.Size([4, 1024, 32, 32])
out3: torch.Size([4, 512, 64, 64])
ConvTransposeBlock
out2: torch.Size([4, 512, 


___

### Part 2

Plot performance over time

Please generate a plot that shows loss on the training set as a function of training time. Make sure your axes are labeled!

**TODO:**

* Plot training loss as function of training time (not Epochs)

**DONE:**


In [0]:
# Your plotting code here


___

### Part 3

Generate a prediction on the pos_test_000072.png image

Calculate the output of your trained network on the pos_test_000072.png image,
then make a hard decision (cancerous/not-cancerous) for each pixel.
The resulting image should be black-and-white, where white pixels represent things
you think are probably cancerous.

**TODO:**

**DONE:**

**NOTE:**

Guessing that the pixel is not cancerous every single time will give you an accuracy of ~ 85%.
Your trained network should be able to do better than that (but you will not be graded on accuracy).
This is the result I got after 1 hour or training.

![](http://liftothers.org/dokuwiki/lib/exe/fetch.php?w=400&tok=d23e0b&media=cs501r_f2016:training_accuracy.png)
![](http://liftothers.org/dokuwiki/lib/exe/fetch.php?w=400&tok=bb8e3c&media=cs501r_f2016:training_loss.png)

In [0]:
# Code for testing prediction on an image
